In [ ]:
pip install --upgrade tensorflow==2.15 transformers rouge_score datasets accelerate>=0.20.1 evaluate


In [ ]:
pip install --upgrade tensorflow-probability


In [ ]:
# pip install --upgrade tensorflow

LOAD DATASET

In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json

--2024-03-11 08:59:17--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34670 (34K) [text/plain]
Saving to: ‘Dataset.json.1’

Dataset.json.1      100%[===================>]  33.86K  --.-KB/s    in 0.002s  

2024-03-11 08:59:17 (13.3 MB/s) - ‘Dataset.json.1’ saved [34670/34670]



In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json

--2024-03-11 08:59:17--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78516 (77K) [text/plain]
Saving to: ‘Train_Dataset.json.1’

Train_Dataset.json. 100%[===================>]  76.68K  --.-KB/s    in 0.01s   

2024-03-11 08:59:17 (7.61 MB/s) - ‘Train_Dataset.json.1’ saved [78516/78516]



In [ ]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (AutoTokenizer, DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer)

In [ ]:
my_dataset_test = load_dataset("json", data_files="Dataset.json", field="documents")

In [ ]:
my_dataset_train = load_dataset("json", data_files="Train_Dataset.json", field="documents")

In [ ]:
my_dataset_train

DatasetDict({
    train: Dataset({
        features: ['TEXT', 'SUMMARIZATION'],
        num_rows: 65
    })
})

PREPROCESS

In [ ]:
checkpoint = "facebook/bart-large-cnn"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["TEXT"]]
    #model_inputs = tokenizer(inputs, padding="max_length", truncation=True, return_tensors="pt")
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    #labels = tokenizer(examples["SUMMARIZATION"], padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(text_target=examples["SUMMARIZATION"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
my_dataset_test = my_dataset_test.map(preprocess_function, batched=True)

In [ ]:
my_dataset_test

DatasetDict({
    train: Dataset({
        features: ['TEXT', 'SUMMARIZATION', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25
    })
})

In [ ]:
my_dataset_train = my_dataset_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

CREATING BATCHING

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

TRAIN

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_controls",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate= False,
    fp16=False,
    push_to_hub=False,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=my_dataset_train["train"],
    eval_dataset=my_dataset_test["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.271205
2,No log,2.312036
3,No log,2.366075


TrainOutput(global_step=195, training_loss=0.9467941675430689, metrics={'train_runtime': 27.2874, 'train_samples_per_second': 7.146, 'train_steps_per_second': 7.146, 'total_flos': 80625179713536.0, 'train_loss': 0.9467941675430689, 'epoch': 3.0})

SAVE MODEL

In [ ]:
model.save_pretrained("my_controls")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
tokenizer.save_pretrained("my_controls")

('my_controls/tokenizer_config.json',
 'my_controls/special_tokens_map.json',
 'my_controls/vocab.json',
 'my_controls/merges.txt',
 'my_controls/added_tokens.json',
 'my_controls/tokenizer.json')

INFERENCE

1 WAY

In [ ]:
text = "summarize: A Standard Change is created from preapproved Standard change templates that have satisfied specific criteria (see Appendix A) and been added to the Standard Change Template Catalog in the Enterprise Ticketing System: the change is repeatable, frequently implemented, is considered low risk and low impact according to the SIA, and has a proven history of success (completed the Normal change lifecycle process at least 3-5 times with no issues). Standard changes that are approved by Division Directors and CAB Co-Chairs are added to the catalog and considered pre-authorized, following a shorter ChM lifecycle outside of the CAB approval process (subject to dual authorization). CRs can request new Standard change templates or use an existing template from the catalog to create a new Standard change request.A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. A Normal change is an RFC that is not a Standard, Expedited or Emergency change, and is subject to the full ChM review process, including review and authorization by the CAB.An Expedited Change does not meet the lead time requirement for a Normal change, but is not an Emergency Change. Service is at risk, although service might not be down, and the RFC needs to be authorized because of a client request that has been validated by SME/ technical expert or a Director, who has determined that that the change needs to go in without waiting for the recommended lead-time. The same ‘Normal’ Change request information is provided in the Enterprise Ticketing System to implement the change, including the reason for expediting the RFC (SIA, back-out plans, scheduled time and downtime required), but lead times are much shorter. Authorization by a CAB Co-Chair is required and Expedited Changes are subject to retroactive review by CAB. Emergency Change is one that must be implemented as soon as possible to correct, or prevent, a high priority incident, or that must be introduced as soon as possible due to likely negative service impacts or situations where the impact to a service is imminent if action is not taken. These changes do not follow the complete lifecycle of a Normal change due to the speed with which they must be implemented and authorized. All emergency changes are authorized by E-CAB and documented and entered into the Enterprise Ticketing System prior to implementation, or as soon as possible after the change has been implemented depending on the nature of the emergency.Emergency changes are subject to a Post Implementation Review (PIR) process by CAB."


In [ ]:
text_prompt ="""A Standard Change is created from preapproved Standard change templates that have satisfied specific criteria (see Appendix A) and been added to the Standard Change Template Catalog in the Enterprise Ticketing System: the change is repeatable, frequently implemented, is considered low risk and low impact according to the SIA, and has a proven history of success (completed the Normal change lifecycle process at least 3-5 times with no issues). Standard changes that are approved by Division Directors and CAB Co-Chairs are added to the catalog and considered pre-authorized, following a shorter ChM lifecycle outside of the CAB approval process (subject to dual authorization). CRs can request new Standard change templates or use an existing template from the catalog to create a new Standard change request.A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. A Normal change is an RFC that is not a Standard, Expedited or Emergency change, and is subject to the full ChM review process, including review and authorization by the CAB.An Expedited Change does not meet the lead time requirement for a Normal change, but is not an Emergency Change. Service is at risk, although service might not be down, and the RFC needs to be authorized because of a client request that has been validated by SME/ technical expert or a Director, who has determined that that the change needs to go in without waiting for the recommended lead-time. The same ‘Normal’ Change request information is provided in the Enterprise Ticketing System to implement the change, including the reason for expediting the RFC (SIA, back-out plans, scheduled time and downtime required), but lead times are much shorter. Authorization by a CAB Co-Chair is required and Expedited Changes are subject to retroactive review by CAB. Emergency Change is one that must be implemented as soon as possible to correct, or prevent, a high priority incident, or that must be introduced as soon as possible due to likely negative service impacts or situations where the impact to a service is imminent if action is not taken. These changes do not follow the complete lifecycle of a Normal change due to the speed with which they must be implemented and authorized. All emergency changes are authorized by E-CAB and documented and entered into the Enterprise Ticketing System prior to implementation, or as soon as possible after the change has been implemented depending on the nature of the emergency.Emergency changes are subject to a Post Implementation Review (PIR) process by CAB.
Write an IT Audit control what to review for the above text
Summary:
"""

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="my_controls")
summarizer(text)

[{'summary_text': 'Review whether the company prioritizes and categorizes changes by whether they are approved by the CAB and if they include the approval of changes that have the support of the CTO, approval of change requests, and testing before implementation in the production environment. The process for approving and implementing changes is based on the individual roles and data classification.'}]

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="my_controls")
summarizer(text_prompt)

[{'summary_text': 'Review whether the company prioritizes and categorizes changes by whether they are approved by the CAB and if they include the approval of changes before implementation in the production environment and verify that they are subject to the strictest standards for approval and testing before implementation.'}]

2 WAY


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_controls")
inputs = tokenizer(text, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_controls")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

tokenizer.decode(outputs[0], skip_special_tokens=True)

'Review whether the company prioritizes and categorizes changes by whether they are approved by the CAB and if they include the submission of change requests, approval of changes, and testing before implementation in the production environment for any changes before implementation and testing prior to testing and validation before implementation on a production environment.'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_controls")
inputs = tokenizer(text_prompt, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_controls")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

tokenizer.decode(outputs[0], skip_special_tokens=True)

'Review whether the company prioritizes and categorizes changes by whether they are approved by the CAB, subject to the approval of a formal process, and if they include the submission of change requests, approval of changes, and testing before implementation in the production environment.'